# 1. Import

In [14]:
import pickle
import pandas as pd
from tqdm import tqdm
import os
import cv2
import numpy as np
import shutil

In [15]:
df_sports = pd.read_csv("../full_metadata_sports.csv")

df_sports

,video_name,action_idx,category,frames,resize,source_video,split,action,new_split
0,01_0001_v_-hyYa8ijq-8_c001.mp4,1,aerobic_gymnastics,31,False,v_-hyYa8ijq-8_c001.mp4,train,explosive push up,train
1,01_0002_v_-hyYa8ijq-8_c001.mp4,1,aerobic_gymnastics,31,True,v_-hyYa8ijq-8_c001.mp4,train,explosive push up,train
2,01_0003_v_-hyYa8ijq-8_c001.mp4,1,aerobic_gymnastics,32,False,v_-hyYa8ijq-8_c001.mp4,train,explosive push up,train
3,05_0001_v_-hyYa8ijq-8_c001.mp4,5,aerobic_gymnastics,89,False,v_-hyYa8ijq-8_c001.mp4,train,support,train
4,05_0002_v_-hyYa8ijq-8_c001.mp4,5,aerobic_gymnastics,87,False,v_-hyYa8ijq-8_c001.mp4,train,support,train
...,...,...,...,...,...,...,...,...,...
41892,mGRum47TLX0_00040.mp4,47,diving,57,True,mGRum47TLX0_00040.mp4,test,Reverse Dive NoTwis TUCK,val
41893,mGRum47TLX0_00044.mp4,47,diving,113,True,mGRum47TLX0_00044.mp4,test,Reverse Dive NoTwis TUCK,test
41894,ovWCmIMMkRI_00050.mp4,47,diving,109,True,ovWCmIMMkRI_00050.mp4,test,Reverse Dive NoTwis TUCK,val
41895,ovWCmIMMkRI_00051.mp4,47,diving,126,True,ovWCmIMMkRI_00051.mp4,test,Reverse Dive NoTwis TUCK,test


In [16]:
len(df_sports[df_sports['frames'] < 32])

20416

# 2. Split video

In [17]:
def check_frame(video_path: str) -> int:
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Cannot open camera")

    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    cap.release()
    cv2.destroyAllWindows()

    return n_frames

In [18]:
def move_files_based_on_df(df: pd.DataFrame, datasets: str):

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        file_name = row['video_name']
        category = row['category']
        destination_subfolder = row['new_split']

        n_frames = check_frame(f"../{datasets}/{category}/{file_name}")

        df.loc[df[df["video_name"] == file_name].index, 'frames'] = n_frames

        source_path = os.path.join(f"../{datasets}/{category}", file_name)
        destination_path = os.path.join(f"../{datasets}/{category}", destination_subfolder)

        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        shutil.move(source_path, destination_path)

    return df

In [19]:
df_sports_2  = move_files_based_on_df(df_sports, datasets="datasets_raw")

100%|██████████| 41897/41897 [02:49<00:00, 246.70it/s]


In [22]:
len(df_sports_2[df_sports_2['frames'] < 32])

20433

In [23]:
df_sports_2.to_csv("../full_metadata_sports.csv",index=False)